In [1]:
import os
import pandas as pd
import numpy as np
import re
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModel,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm

In [2]:
train_file = "/content/Train_data.txt"
test_file = "/content/Test_data.txt"

In [3]:
def read_data(file):
    data = []
    with open(file, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(line.strip())
    return data

In [4]:
train_data_preprocess = read_data(train_file)
test_data_preprocess = read_data(test_file)

In [5]:
df_train = pd.DataFrame(train_data_preprocess, columns=['text'])
df_test = pd.DataFrame(test_data_preprocess, columns=['text'])

df_train['label'] = df_train['text'].apply(lambda x: x.split()[0].replace('__label__', ''))
df_train['text'] = df_train['text'].apply(lambda x: ' '.join(x.split(' ')[1:]))

df_test['label'] = df_test['text'].apply(lambda x: x.split(' ')[0].replace('__label__', ''))
df_test['text'] = df_test['text'].apply(lambda x: ' '.join(x.split(' ')[1:]))

df_train

,text,label
0,"Theo hành trình tour du lịch Mỹ - Bờ Đông, du ...",Du_lich
1,mình cần tìm 1 phòng cho khoảng 3 người quanh...,Nha_dat
2,Cho thuê nhà riêng dt 60m/sàn. Có 4 phòng ngủ...,Nha_dat
3,"Cho thuê nhà ở tầng 4 khép kín, 4/295 Nguyễn K...",Nha_dat
4,► Crumpler jackpack full photo ► giá : 800.000...,Mua_sam
...,...,...
15995,CÁC MÓN KIM CHI NGON CHO MÙA THU -------------...,Do_an_va_do_uong
15996,Cần cho thuê Chung cư Greenstar 234 Phạm Văn Đ...,Nha_dat
15997,CHƯƠNG TRÌNH HỌC PHÍ THÁNG 08/2016 TẶNG NGAY ...,Kinh_doanh_va_Cong_nghiep
15998,Bố trí thông minh giúp nhà ống Sài Gòn không c...,Nha_va_vuon


In [6]:
df_test

,text,label
0,Gấp ; Hiện bên em đang cần thuê 1 phòng có Diệ...,﻿Nha_dat
1,🌈 CHÀO NOEL ĐÓN MƯA QUÀ TẶNG . 😍 Nhân dịp Noel...,Mang_internet_va_vien_thong
2,📢📢📢 KHỞI CÔNG XÂY DỰNG 33 CĂN NHÀ PHỐ LIỀN KỀ ...,Kinh_doanh_va_Cong_nghiep
3,"Sáng ngày hôm nay, BTC rất vui khi nhận được s...",Sach
4,Cần cho thuê căn hộ chung cư dưới sài đồng đối...,Nha_dat
...,...,...
10012,[ TỔNG HỢP NHỮNG MÓN NGON KHU VỰC ĐÀO TẤN - BA...,Do_an_va_do_uong
10013,Bản tin tài chính kinh doanh tối thứ sáu (23/0...,Tai_chinh
10014,"Ngang nhiên vừa hack vừa stream, game thủ Over...",Giai_tri
10015,"5 TOUR NƯỚC NGOÀI DỊP GIÁNG SINH, NĂM MỚI GIÁ ...",Du_lich


# Data Processing

In [7]:
def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^\w\s]|_', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()

    return text

In [8]:
test = preprocess_text('Đau mỏi vai gáy nên đi massage cổ, nhưng sau đó nữ ca sĩ đã không thể di chuyển vì gặp nhiều triệu chứng đau nhức, không thể nằm thẳng lưng được. Sau gần 1 tháng điều trị, cô đã không thể qua khỏi. Theo: Vietnamnet')
test

'đau mỏi vai gáy nên đi massage cổ nhưng sau đó nữ ca sĩ đã không thể di chuyển vì gặp nhiều triệu chứng đau nhức không thể nằm thẳng lưng được sau gần tháng điều trị cô đã không thể qua khỏi theo vietnamnet'

In [9]:
def preprocess_data(df):
    df['text'] = df['text'].apply(lambda x: preprocess_text(x))
    df.drop_duplicates(subset=['text'], inplace=True)
    return df

In [10]:
df_train = preprocess_data(df_train)
df_test = preprocess_data(df_test)

In [11]:
print(df_train['text'][:5])
print(df_test['text'][:5])

0    theo hành trình tour du lịch mỹ bờ đông du khá...
1    mình cần tìm phòng cho khoảng người quanh khu ...
2    cho thuê nhà riêng dt msàn có phòng ngủ p khác...
3    cho thuê nhà ở tầng khép kín nguyễn khoái có b...
4    crumpler jackpack full photo giá vnđ giảm còn ...
Name: text, dtype: object
0    gấp hiện bên em đang cần thuê phòng có diện tí...
1    chào noel đón mưa quà tặng nhân dịp noel viett...
2    khởi công xây dựng căn nhà phố liền kề chỉ tri...
3    sáng ngày hôm nay btc rất vui khi nhận được sá...
4    cần cho thuê căn hộ chung cư dưới sài đồng đối...
Name: text, dtype: object


In [12]:
print(df_train.dtypes)
print(df_test.dtypes)

text     object
label    object
dtype: object
text     object
label    object
dtype: object


In [13]:
label_to_encoded = {label: idx for idx, label in enumerate(df_train['label'].unique())}

df_train['label'] = df_train['label'].map(label_to_encoded)
df_test['label'] = df_test['label'].map(label_to_encoded)

In [14]:
label_to_encoded

{'Du_lich': 0,
 'Nha_dat': 1,
 'Mua_sam': 2,
 'Tai_chinh': 3,
 'Mang_internet_va_vien_thong': 4,
 'Nha_va_vuon': 5,
 'Kinh_doanh_va_Cong_nghiep': 6,
 'Nghe_thuat': 7,
 'Giao_duc': 8,
 'Lam_dep_va_the_hinh': 9,
 'Con_nguoi_va_xa_hoi': 10,
 'Sach': 11,
 'Chinh_tri': 12,
 'Do_an_va_do_uong': 13,
 'Giao_thong': 14,
 'Thoi_quen_va_so_thich': 15,
 'Giai_tri': 16,
 'Suc_khoe_va_benh_tat': 17,
 'Phap_luat': 18,
 'Khoa_hoc': 19,
 'May_tinh_va_thiet_bi_dien_tu': 20,
 'Cong_nghe_moi': 21,
 'The_thao': 22}

In [15]:
label_to_encoded_format = {
    'Du lịch': 0,
    'Nhà đất': 1,
    'Mua sắm': 2,
    'Tài chính': 3,
    'Mạng internet và viễn thông': 4,
    'Nhà và vườn': 5,
    'Kinh doanh và công nghiệp': 6,
    'Nghệ thuật': 7,
    'Giáo dục': 8,
    'Làm đẹp và thể hình': 9,
    'Con người và xã hội': 10,
    'Sách': 11,
    'Chính trị': 12,
    'Đồ ăn và đồ uống': 13,
    'Giao thông': 14,
    'Thói quen và sở thích': 15,
    'Giải trí': 16,
    'Sức khoẻ và bệnh tật': 17,
    'Pháp luật': 18,
    'Khoa học': 19,
    'Máy tính và thiết bị điện tử': 20,
    'Công nghệ mới': 21,
    'Thể thao': 22
}

In [16]:
print(df_train[:5])
print(df_test[:5])

                                                text  label
0  theo hành trình tour du lịch mỹ bờ đông du khá...      0
1  mình cần tìm phòng cho khoảng người quanh khu ...      1
2  cho thuê nhà riêng dt msàn có phòng ngủ p khác...      1
3  cho thuê nhà ở tầng khép kín nguyễn khoái có b...      1
4  crumpler jackpack full photo giá vnđ giảm còn ...      2
                                                text  label
0  gấp hiện bên em đang cần thuê phòng có diện tí...    NaN
1  chào noel đón mưa quà tặng nhân dịp noel viett...    4.0
2  khởi công xây dựng căn nhà phố liền kề chỉ tri...    6.0
3  sáng ngày hôm nay btc rất vui khi nhận được sá...   11.0
4  cần cho thuê căn hộ chung cư dưới sài đồng đối...    1.0


In [17]:
print("Số lượng NaN trong df_train:")
print(df_train.isnull().sum())
print("\nSố lượng NaN trong df_test:")
print(df_test.isnull().sum())

Số lượng NaN trong df_train:
text     0
label    0
dtype: int64

Số lượng NaN trong df_test:
text     0
label    1
dtype: int64


In [18]:
df_test = df_test.iloc[1:].reset_index(drop=True)

In [19]:
df_test['label'] = df_test['label'].astype(int)

In [20]:
print(df_test.head())

                                                text  label
0  chào noel đón mưa quà tặng nhân dịp noel viett...      4
1  khởi công xây dựng căn nhà phố liền kề chỉ tri...      6
2  sáng ngày hôm nay btc rất vui khi nhận được sá...     11
3  cần cho thuê căn hộ chung cư dưới sài đồng đối...      1
4  bài dự thi của ban nhạc old mac donal band ban...      7


In [21]:
print(df_train.dtypes)
print(df_test.dtypes)

text     object
label     int64
dtype: object
text     object
label     int64
dtype: object


In [ ]:
def save_csv_data(df, path):
  df.to_csv(path, encoding='utf-8')

# Finetune phoBERT

In [22]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [23]:
class VietnameseTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        # Tokenize
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [24]:
class PhoBERTClassifier(nn.Module):
    def __init__(self, model_name, num_classes, dropout_rate=0.3):
        super(PhoBERTClassifier, self).__init__()
        self.phobert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.phobert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        output = self.dropout(pooled_output)
        return self.classifier(output)

In [25]:
print("Preparing datasets...")
train_texts = df_train['text'].tolist()
train_labels = df_train['label'].tolist()
test_texts = df_test['text'].tolist()
test_labels = df_test['label'].tolist()

train_dataset = VietnameseTextDataset(train_texts, train_labels, tokenizer)
test_dataset = VietnameseTextDataset(test_texts, test_labels, tokenizer)

BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Preparing datasets...


In [26]:
# Training function
def train_epoch(model, data_loader, optimizer, scheduler, criterion, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    progress_bar = tqdm(data_loader, desc="Training")

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{correct_predictions/total_predictions:.4f}'
        })

    return total_loss / len(data_loader), correct_predictions / total_predictions

In [27]:
def evaluate(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)

            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    return total_loss / len(data_loader), accuracy, predictions, true_labels

In [30]:
NUM_CLASSES = len(label_to_encoded)
MODEL_NAME = "vinai/phobert-base"
model = PhoBERTClassifier(MODEL_NAME, NUM_CLASSES)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

# Optimizer và scheduler
LEARNING_RATE = 2e-5
EPOCHS = 3
WARMUP_STEPS = 100

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=total_steps
)

# Loss function
criterion = nn.CrossEntropyLoss()

In [31]:
# Training loop
print("Starting training...")
best_accuracy = 0
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(EPOCHS):
    print(f'\nEpoch {epoch + 1}/{EPOCHS}')
    print('-' * 50)

    # Training
    train_loss, train_acc = train_epoch(
        model, train_loader, optimizer, scheduler, criterion, device
    )

    # Evaluation
    val_loss, val_acc, predictions, true_labels = evaluate(
        model, test_loader, criterion, device
    )

    # Store metrics
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    # Save best model
    if val_acc > best_accuracy:
        best_accuracy = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_accuracy': best_accuracy,
            'label_to_encoded': label_to_encoded_format
        }, 'best_phobert_classifier.pth')
        print(f'New best model saved with accuracy: {best_accuracy:.4f}')

print(f'\nTraining completed! Best validation accuracy: {best_accuracy:.4f}')

Starting training...

Epoch 1/3
--------------------------------------------------


Evaluating: 100%|██████████| 603/603 [02:10<00:00,  4.62it/s]


Train Loss: 0.9641, Train Acc: 0.7751
Val Loss: 0.3821, Val Acc: 0.8802
New best model saved with accuracy: 0.8802

Epoch 2/3
--------------------------------------------------


Evaluating: 100%|██████████| 603/603 [02:10<00:00,  4.63it/s]


Train Loss: 0.3020, Train Acc: 0.9009
Val Loss: 0.3219, Val Acc: 0.8830
New best model saved with accuracy: 0.8830

Epoch 3/3
--------------------------------------------------


Evaluating: 100%|██████████| 603/603 [02:18<00:00,  4.36it/s]


Train Loss: 0.2204, Train Acc: 0.9213
Val Loss: 0.3036, Val Acc: 0.8898
New best model saved with accuracy: 0.8898

Training completed! Best validation accuracy: 0.8898


In [32]:
# Tạo classification report
encoded_to_label = {v: k for k, v in label_to_encoded_format.items()}
target_names = [encoded_to_label[i] for i in range(len(label_to_encoded_format))]

print("\nClassification Report:")
print(classification_report(true_labels, predictions, target_names=target_names))


Classification Report:
                              precision    recall  f1-score   support

                     Du lịch       0.96      0.98      0.97       604
                     Nhà đất       0.98      0.99      0.98      1583
                     Mua sắm       0.95      0.95      0.95       754
                   Tài chính       0.54      0.29      0.37       758
 Mạng internet và viễn thông       0.99      0.98      0.98       396
                 Nhà và vườn       0.97      0.91      0.94       158
   Kinh doanh và công nghiệp       0.62      0.79      0.70      1250
                  Nghệ thuật       0.99      0.99      0.99       398
                    Giáo dục       0.96      0.94      0.95       460
         Làm đẹp và thể hình       0.95      0.96      0.95       181
         Con người và xã hội       0.92      0.94      0.93       204
                        Sách       0.94      0.96      0.95       245
                   Chính trị       0.97      0.98      0.97      

In [33]:
def predict_text(model, tokenizer, text, label_to_encoded, device, max_length=256):
    model.eval()

    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = torch.nn.functional.softmax(outputs, dim=-1)
        predicted_class_id = torch.argmax(probabilities, dim=-1).item()
        confidence = probabilities[0][predicted_class_id].item()

    encoded_to_label = {v: k for k, v in label_to_encoded.items()}
    predicted_label = encoded_to_label[predicted_class_id]

    return predicted_label, confidence

In [35]:
test_text = "Tôi muốn mua một chiếc điện thoại mới với camera tốt"
test_text_preprocessed = preprocess_text(test_text)
predicted_label, confidence = predict_text(
    model, tokenizer, test_text_preprocessed, label_to_encoded_format, device
)
print(f"\nSample prediction:")
print(f"Text: {test_text}")
print(f"Predicted label: {predicted_label}")
print(f"Confidence: {confidence:.4f}")


Sample prediction:
Text: Tôi muốn mua một chiếc điện thoại mới với camera tốt
Predicted label: Mạng internet và viễn thông
Confidence: 0.7746
